In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import QuantileTransformer,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import joblib
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
df=pd.read_csv("music_genre.csv")

In [3]:
df.dropna()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50000,58878.0,BEXEY,GO GETTA,59.0,0.03340,0.913,-1.0,0.574,0.00000,C#,0.119,-7.022,Major,0.2980,98.02799999999999,4-Apr,0.330,Hip-Hop
50001,43557.0,Roy Woods,Drama (feat. Drake),72.0,0.15700,0.709,251860.0,0.362,0.00000,B,0.109,-9.814,Major,0.0550,122.04299999999999,4-Apr,0.113,Hip-Hop
50002,39767.0,Berner,Lovin' Me (feat. Smiggz),51.0,0.00597,0.693,189483.0,0.763,0.00000,D,0.143,-5.443,Major,0.1460,131.079,4-Apr,0.395,Hip-Hop
50003,57944.0,The-Dream,Shawty Is Da Shit,65.0,0.08310,0.782,262773.0,0.472,0.00000,G,0.106,-5.016,Minor,0.0441,75.88600000000001,4-Apr,0.354,Hip-Hop


In [4]:
X=(df.drop(columns=['music_genre']).dropna()).values

In [5]:
y=df['music_genre'].dropna()

In [6]:
le=LabelEncoder()
for i in range(len(X[0])):
    X[:,i]=le.fit_transform(X[:,i])
# for i in range(len(y[0])):
#     y[:,i]=le.fit_transform(y[:,i])  
#     y=y.astype('int') 

In [7]:
print('X',X.shape)
print('Y',y.shape)

X (50000, 17)
Y (50000,)


In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
pipe=Pipeline([
    ('scale',QuantileTransformer()),
    ('model',svm.SVC())
])
pipe.fit(X_train,y_train)
# pipe.get_params()
ppred=pipe.predict(X_test)
print('Actual:',y_test)
print('Predicted:',ppred)
print('Score:',accuracy_score(y_test,ppred))

Actual: 1606      Electronic
44796      Classical
9699           Anime
22881        Country
11345           Jazz
            ...     
11259           Jazz
11052           Jazz
2793      Electronic
18448    Alternative
38678           Rock
Name: music_genre, Length: 10000, dtype: object
Predicted: ['Electronic' 'Classical' 'Electronic' ... 'Anime' 'Alternative' 'Hip-Hop']
Score: 0.5851


In [ ]:
grid=GridSearchCV(
estimator=pipe,
param_grid={'model__random_state':[1,2,3,4,5]},
cv=10)
grid.fit(X_train,y_train)
pred=grid.predict(X_test)

In [73]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.783995,0.013772,0.015701,0.002831,1,{'model__random_state': 1},0.44675,0.46750,0.45875,0.46900,0.45475,0.46000,0.44550,0.45300,0.45600,0.45850,0.456975,0.007266,1
1,0.784299,0.012336,0.015310,0.000638,2,{'model__random_state': 2},0.45300,0.46025,0.45125,0.46425,0.46000,0.45475,0.45550,0.44725,0.45525,0.45725,0.455875,0.004626,3
2,0.781901,0.021290,0.014904,0.000302,3,{'model__random_state': 3},0.44850,0.46625,0.45375,0.45875,0.44900,0.46025,0.45675,0.45025,0.45775,0.45700,0.455825,0.005276,4
3,0.777603,0.009566,0.014700,0.000460,4,{'model__random_state': 4},0.45300,0.46475,0.45250,0.46675,0.45425,0.45425,0.44925,0.45150,0.44950,0.45850,0.455425,0.005749,5
4,0.779002,0.010890,0.015000,0.000448,5,{'model__random_state': 5},0.44800,0.46625,0.46175,0.46625,0.45375,0.45625,0.44750,0.45275,0.45725,0.45975,0.456950,0.006338,2
